In [184]:
import warnings
import json
import os
import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")
openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

In [185]:
def title_json_data(json_files):
    """
    여러 JSON 파일을 읽고 데이터를 통합한 후 특정 형식의 문자열 리스트로 반환

    Parameters:
        json_files (list): JSON 파일 경로 리스트

    Returns:
        list: 파일 데이터에서 'title'과 'content'를 읽어 특정 형식으로 변환한 리스트
    """
    all_json_data = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_json_data.extend(data)
        except FileNotFoundError:
            print(f"Error: 파일을 찾을 수 없습니다 - {file_path}")
        except json.JSONDecodeError:
            print(f"Error: JSON 파일 형식이 잘못되었습니다 - {file_path}")

    # 'title'과 'content'를 읽어 특정 형식으로 반환
    title = [item.get('title', 'N/A') for item in all_json_data]
    return title

In [186]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

def load_vector_store(db_name: str, DB_PATH):
        """
        로컬에 저장된 크로마 db를 불러옴

        Parameters:
            db_name : db 생성시 설정한 이름
            DB_PATH : db 경로
        Returns:
            Chroma 객체
        """
        return Chroma(
        collection_name=db_name,
        persist_directory=DB_PATH,
        embedding_function=OpenAIEmbeddings(
            model="text-embedding-ada-002", api_key=openai.api_key
        ),
    )

### 데이터 셋에서 제목만 추출
- 제목 리스트로 무작위 선택

In [187]:
json_files = ['./documents/filtered_unsolved_cases.json', './documents/korea_crime.json']
titles = title_json_data(json_files)
sample_titles = titles[0:11]

### 미리 제작된 스크립트가 저장되어 있는 db 불러오기

In [188]:
path = './db/script_db'
db_name = 'script_db'
script_db = load_vector_store(db_name, path)

In [189]:
rt = script_db.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print(rt.invoke('정인숙')[0].page_content)

# 제목: 정인숙 피살사건

# 프롤로그 1: 사건을 다룬 미디어
- 2010년 3월 20일, SBS TV에서 방영된 프로그램은 정인숙 피살사건의 실체를 추적하는 내용을 담고 있다. 이 프로그램은 사건의 복잡성과 미해결 상태를 조명하며, 당시 수사 기록과 현장 감식 자료를 공개하여 새로운 시각을 제공하였다.

# 프롤로그 2: 사건의 배경
- 정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 살인 사건이다. 이 사건은 고급 호스티스에서 일하던 정인숙이 총에 맞아 사망한 사건으로, 그녀의 형 정종욱이 부상을 입고 생존하였다. 사건은 정인숙의 자녀 아버지가 고위 정치인이라는 점에서 정부의 개입 의혹을 불러일으켰다.

# 프롤로그 3: 정인숙의 인물상
- 정인숙은 1945년 1월 1일에 태어나 고위 공직자 가문에서 자랐다. 그녀는 대학을 중퇴하고 고급 호스티스로 일하며 사회의 주목을 받았다. 정인숙은 당시 26세였고, 3세 아들을 두고 있었다. 그녀의 소지품에서 고위 인사들의 명함이 발견되었고, 숨겨진 자녀에 대한 소문이 돌았다.

# 전개 1: 사건의 역사적 배경
- 1970년대 초반, 한국은 정치적 불안정과 사회적 갈등이 심화되던 시기였다. 이 시기에 발생한 정인숙 피살사건은 고위층의 부패와 권력 남용에 대한 의혹을 불러일으켰다. 사건은 당시 사회의 불신과 불안감을 더욱 부각시켰다.

# 전개 2: 정인숙의 개인사
- 정인숙은 고위 공무원의 딸로 태어나 여러 명의 친오빠가 있었다. 그녀는 고급 요정에서 호스티스로 일하며, 당시 한일회담이 이루어진 선운각 등에서 활동하였다. 정인숙은 당시 정부의 유력 인사와의 관계로 인해 사회적 스캔들의 중심에 서게 되었다.

# 전개 3: 사건의 시작
- 1970년 3월 17일 밤 11시경, 서울 마포구 합정동 절두산 근처 도로에서 정인숙과 그녀의 형 정종욱이 탄 검은색 코로나 승용차가 멈춰 서 있었다. 이때 정인숙은 총에 맞아 사망하고, 정종욱은 부상을 입었다. 사건은 교통사고로 위장되었으나, 이후 총

### 검증 LLM
- 사용자의 질문과 RAG를 통해 검색된 스크립트의 연관성을 검증
- 스크립트의 전체 맥락을 고려하여 사용자가 찾는 이야기가 맞는지 판단
- 정수형 점수 반환

In [190]:
def evaluator(query, db):
    """
    db에서 찾아온 스크립트가 적절한지 판단하는 함수

    Parameters:
        query : 사용자 입력
        db : 스크립트가 저장된 db
    Returns:
        연관 정도 점수
        스크립트 : 연관 정도가 적절한 경우
    """
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=openai.api_key,
        max_tokens=100,
        temperature=0.0,
    )
    script_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})
    script = script_retriever.invoke(query)[0].page_content
    prompt = ChatPromptTemplate.from_template(
    """
    persona : relavence check machine
    **return only integer score**
    1. extract subject of script
    2. check relavence between query and subject
    3. calculate elaborate score 
    4. maximum '100', minimum '0', 
    5. increas by '5'
    6. sample is about conversation
    <sample>
    script : 'title : 강다니엘 이모 사건, content : 나 아는사람 강다니엘 닮은 이모가 다시보게되는게 다시 그때처럼 안닮게 엄마보면 느껴지는걸수도 있는거임?'

    query : '사건'
    ai : '10'

    query : '이모'
    ai : '25'

    query : '이모 사건'
    ai : '80'

    query : '강다니엘 사건'
    ai : '85'

    query : '강다니엘 이모'
    ai : '95'
    </sample>

    <query>
    {query}
    </query>

    <script>
    {script}
    </script>
    """
    )
    chain = prompt | llm | StrOutputParser()
    score = chain.invoke({"query": query, 'script' : script})
    if not score : return [0, 'N/A']
    return [int(score), script]

### 프롬프트 수정
#### 기존
- 사용자 입력마다 스크립트를 불러옴
- 대부분 대화 메모리로 맥락을 유지하여 같은 스크립트를 불러옴
- 일정 확률로 다른 스크립트를 불러오는 문제 발생

### 수정 
- 검증 LLM 으로 검증한 스크립트로 고정
- `RunnableMap` : 인자 설정이 자유로운 chain 형식
- 일단 시작된 이야기는 그대로 유지

In [191]:
from langchain_core.prompts import PromptTemplate
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableMap

openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

def chain_maker(script):
    """
    스크립트를 바탕으로 대화를 이어나가는 llm chain 생성

    Parameters:
        script : 선택된 스크립트
    Returns:
        llm chain
     """
    prompt = PromptTemplate.from_template(
    """
    persona : story teller
    language : only korean
    tell dramatic story like talking to friend,
    speak informally,
    progress chapter by chapter,
    **hide header like '###'**,
    start chapter with interesting question,
    wait user answer,
    give reaction to answer,
    do not use same reaction or same question
    
    # script
    {script}

    #Previous Chat History:
    {chat_history}

    #Question: 
    {question} 
    """
    )

    llm = ChatOpenAI(model="gpt-4o-mini", api_key= openai.api_key, temperature=0.3)

    # 단계 8: 체인(Chain) 생성
    chain = RunnableMap(
        {
            "script": lambda inputs: script,  # script는 고정값으로 전달
            "question": itemgetter("question"),  # 입력에서 question 추출
            "chat_history": itemgetter("chat_history"),  # 입력에서 chat_history 추출
        }
    ) | prompt | llm | StrOutputParser()
    return chain

def history_chain(chain, memory_store : dict):
    """
    맥락을 유지하면 대화하는 chain 생성

    Parameters:
        chain : script 를 찾아 답변하는 chain
        memory_store : 대화의 맥락이 저장될 공간
    Returns:
        memory history chain
     """
    def get_session_history(session_ids):
        print(f"[대화 세션ID]: {session_ids}")
        if session_ids not in memory_store:  # 세션 ID가 store에 없는 경우
            # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
            memory_store[session_ids] = ChatMessageHistory()
        return memory_store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


    # 대화를 기록하는 RAG 체인 생성
    rag_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
        history_messages_key="chat_history",  # 기록 메시지의 키
    )
    return rag_with_history

In [192]:
from langchain.memory import ConversationSummaryMemory

def documents_filter(SPLITS):
    """
    분할된 데이터에서 불필요한 데이터를 제거하고 하나로 결합
    ConversationSummaryMemory에 이전 내용을 요약하여 저장
    아전 내용과 대조해서 불필요한 데이터 구분

    Parameters:
        SPLITS: 분할된 텍스트 데이터 : Document

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
                model="gpt-4o-mini",
                api_key=openai.api_key,
                max_tokens=1000,
                temperature=0.0,
            )
    summaries = []
    memory = ConversationSummaryMemory(
        llm=llm, return_messages=True)
    
    count = 0
    for SPLIT in SPLITS:
        SPLIT = SPLIT.page_content

        try:
            context = memory.load_memory_variables({})["history"]
            prompt = ChatPromptTemplate.from_template(
                """
                persona : documents filter
                language : only in korean
                extract the parts related to the context and ignore the rest,
                write blanck if it's not relevant,
                
                <context>
                {context}
                </context>
                
                <docs>
                {SPLIT}
                </docs>
                """
            )
            chain = prompt | llm | StrOutputParser()
            summary = chain.invoke({"SPLIT": SPLIT, 'context' : context})
            memory.save_context({"input": f'summary # {count}'}, {"output": summary})
            summaries.append(summary)
            count+=1

        except Exception as e:
            # 오류 처리: 만약 API 호출 중에 문제가 발생하면 오류 메시지 추가
            print(f"Error summarizing document: {e}")
            summaries.append(f"Error summarizing document: {e}")

    return "".join(summaries)

def generate_script(summaries):
    """
    대화 LLM이 전달할 이야기의 대본 생성

    Parameters:
        summaries: 필터링된 텍스트 데이터

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=openai.api_key,
        max_tokens=5000,
        temperature=0.0,
    )
    prompt = ChatPromptTemplate.from_template(
    """
    persona = script writer
    language = only in korean
    least 3000 tokens
    use input,
    refer to sample,
    write about time, character, event,
    write only fact
    ignore the mere listing of facts and write N/A
 
    <sample>
    # title : title of script
    # prologue 1 : song, movie, book, show about subject
    - coontent :
    # prologue 2 : explain about subject
    - coontent :
    # prologue 3 : explain about character
    - coontent :
    # exposition 1 : historical background of subject
    - coontent :
    # exposition 2 : history of character
    - coontent :
    # exposition 3 : beginning of event
    - coontent :
    # development 1 : situation, action, static of character
    - coontent :
    # development 2 : influence of event
    - coontent :
    # development 3 : reaction of people
    - coontent :
    # climax 1 : event and effect bigger
    - coontent :
    # climax 2 : dramatic action, conflict
    - coontent :
    # climax 3 : falling Action
    - coontent :
    # denouement : resolution
    - coontent :
    # epilogue : message, remaining
    - coontent :
    </sample>

    <input>
    {summaries}
    </input>

    """
    )
    chain = prompt | llm | StrOutputParser()
    script = chain.invoke({"summaries": summaries})
    return script

### 사용자가 입력한 정보로 새로운 스크립트 생성

In [193]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document

def script_maker(INPUT : str):
  print("다소 시간이 소요될 수 있습니다.")
  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=100
        )
  if INPUT.startswith("http"):
        url = INPUT
        web_docs = WebBaseLoader(url).load()
        if web_docs[0].metadata['title'] : title = web_docs[0].metadata['title']
        else : title = ''
        docs = f"title : {title} \n\n" + web_docs[0].page_content
  else:
        docs= str(INPUT)
  documents = [Document(page_content=docs)]
  SPLITS = text_splitter.split_documents(documents)
  refined = documents_filter(SPLITS)
  return generate_script(refined)
  

In [194]:
url = 'https://namu.wiki/w/77246%20위조지폐%20유통사건'

### 검증 LLM이 반환안 점수에 따라 진행 결정
- 80 점 미만 : 종료, 재시작, 생성
- 80-95점 : 자세한 설명 요구
- 95점 : 그대로 진행

#### 생성
- 스크립트 생성 함수로 새로운 스크립트 생성
- 스크립트 db에 저장
- 다시 답변 요구

In [195]:
import random
while True:
    print("================================")
    path = './db/script_db'
    db_name = 'script_db'
    script_db = load_vector_store(db_name, path)
    query = input("어떤 이야기가 듣고 싶으신가요?")
    print(query)
    if query.lower() == "exit":
        print("대화를 종료합니다.")
        query = False
        break
    elif query is None or  "아무거나" in query.strip():
        print("재미난 이야기를 가져오는 중...")
        choice = random.choice(sample_titles)
        query = choice
        print(choice)
        break
    
    relavence = evaluator(query, script_db)
    print(relavence[0])
    if relavence[0] < 80: 
         print('모르는 이야기 입니다.', '종료 : exit', '다시 물어보기 : return', '생성하기 : create')
         user_input = input('입력하세요.')
         if user_input.lower() == "exit":
            print("대화를 종료합니다.")
            query = False
            break
         elif user_input.lower() == "return":
            continue
         
         elif user_input.lower() == "create":
            text_input = input('URL 또는 텍스트를 입력해주세요.')
            new_script = script_maker(text_input)
            script_documents = [
                Document(page_content=new_script),
             ]
            script_db.add_documents(script_documents)
            script_db.persist()
            print('생성이 완료되었습니다.', '다시 답변해주세요.')
            continue
         
    elif relavence[0] < 95 and relavence[0] >= 80:
        print("더 자세히 이야기 해주세요")
        continue
    elif relavence[0] >= 95:
        script = relavence[1]
        break

if query : 
    store ={}
    chain = chain_maker(script)
    h_chain = history_chain(chain, store)

    response = h_chain.invoke(
        # 질문 입력
        {"question": query},
        # 세션 ID 기준으로 대화를 기록합니다.
        config={"configurable": {"session_id": "test21"}},
    )
    print("\n답변:")
    print(response)

    while True:
        print("========================")
        query = input("반응을 입력하세요.")
        if query.lower() == "exit":
                print("대화를 종료합니다.")
                break
        response = h_chain.invoke(
        # 질문 입력
        {"question": query},
        # 세션 ID 기준으로 대화를 기록합니다.
        config={"configurable": {"session_id": "test21"}},
    )
        print(query)
        print("\n답변:")
        print(response)

777
85
더 자세히 이야기 해주세요
777 항공편
85
더 자세히 이야기 해주세요
대한항공 77
85
더 자세히 이야기 해주세요
대한항공 777편
85
더 자세히 이야기 해주세요
대한항공 격추
95
[대화 세션ID]: test21

답변:
그 사건에 대해 들어본 적 있어? 대한항공 007편 격추 사건 말이야. 정말 충격적이고 드라마틱한 이야기야. 너는 어떻게 생각해?
대화를 종료합니다.


In [196]:
print(store['test21'])

Human: 대한항공 격추
AI: 그 사건에 대해 들어본 적 있어? 대한항공 007편 격추 사건 말이야. 정말 충격적이고 드라마틱한 이야기야. 너는 어떻게 생각해?


### 추가할 사항
- 다국어 기능
- 코드 정리